# 深度学习方法

In [127]:
import os
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation

In [2]:
os.listdir('.')

['predict.py',
 '.gitignore',
 'deep.ipynb',
 '.git',
 'notebook.ipynb',
 'dataset',
 '.idea']

In [75]:
baseinfo = pd.read_csv('dataset/1baseinfo.csv')

train_labels = pd.read_csv('dataset/train.csv')




,EID,CreateYear,TradeType,RegisteredCapital,Type,Feature1,Feature2,Feature3,Feature4,Feature5,Y,EndDate
0,732046267,1980,51,480.0,13,25.0,1.0,98041.0,17.0,0.0,0,197012
1,1338267101,2005,74,240.0,5,0.0,2.0,1500.0,0.0,0.0,0,197012
2,1057042610,2011,75,3000.0,6,4.0,8.0,10500.0,13.0,0.0,0,197012
3,400121247,1982,55,1000.0,6,9.0,2.0,619318.0,31.0,0.0,0,197012
4,970179772,1986,53,150.0,17,4.0,1.0,13000.0,31.0,0.0,1,201612


In [96]:
baseinfo['EID'] = baseinfo['EID'].fillna(0)
baseinfo['CreateYear'] = baseinfo['CreateYear'].fillna(0)
baseinfo['TradeType'] = baseinfo['TradeType'].fillna(0).astype('int')
baseinfo['RegisteredCapital'] = baseinfo['RegisteredCapital'].fillna(0)
baseinfo['Feature1'] = baseinfo['Feature1'].fillna(0)
baseinfo['Feature2'] = baseinfo['Feature2'].fillna(0)
baseinfo['Feature3'] = baseinfo['Feature3'].fillna(0)
baseinfo['Feature4'] = baseinfo['Feature4'].fillna(0)
baseinfo['Feature5'] = baseinfo['Feature5'].fillna(0)
train_labels['EndDate'] = train_labels['EndDate'].fillna('197012').astype('int')

merged_data = pd.merge(baseinfo, train_labels, on='EID', how='right')
merged_data.head()


,EID,CreateYear,TradeType,RegisteredCapital,Type,Feature1,Feature2,Feature3,Feature4,Feature5,Y,EndDate
0,732046267,1980,51,480.0,13,25.0,1.0,98041.0,17.0,0.0,0,197012
1,1338267101,2005,74,240.0,5,0.0,2.0,1500.0,0.0,0.0,0,197012
2,1057042610,2011,75,3000.0,6,4.0,8.0,10500.0,13.0,0.0,0,197012
3,400121247,1982,55,1000.0,6,9.0,2.0,619318.0,31.0,0.0,0,197012
4,970179772,1986,53,150.0,17,4.0,1.0,13000.0,31.0,0.0,1,201612


In [157]:
## split data to train and test
count = len(merged_data)


all_x = merged_data.iloc[:,1:10]
all_x = all_x.values

all_y = merged_data.iloc[:,10:11]
all_y = all_y.values



In [158]:
print(type(all_y),type(all_x))

all_x = sequence.pad_sequences(all_x)
all_y = sequence.pad_sequences(all_y)

print(all_x.shape)
print(all_y.shape)


<class 'numpy.ndarray'> <class 'numpy.ndarray'>


(150095, 9)
(150095, 1)


In [ ]:
model = Sequential()

model.add(Dense(1,input_dim = 9))
model.add(Activation('relu'))
Dense(1)


# 二分类问题
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])




model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(all_x, all_y,
          validation_split = 0.09,
          batch_size=32,
          epochs=2,
          )


Train on 135085 samples, validate on 15010 samples
Epoch 1/2


    32/135085 [..............................] - ETA: 22:28 - loss: 14.4478 - acc: 0.0000e+00

  1120/135085 [..............................] - ETA: 44s - loss: 13.1246 - acc: 0.0384      

  2176/135085 [..............................] - ETA: 25s - loss: 13.1150 - acc: 0.0391

  3296/135085 [..............................] - ETA: 18s - loss: 13.0747 - acc: 0.0391

...

130624/135085 [============================>.] - ETA: 0s - loss: 13.3171 - acc: 0.0329

131840/135085 [============================>.] - ETA: 0s - loss: 13.3183 - acc: 0.0329

133088/135085 [============================>.] - ETA: 0s - loss: 13.3171 - acc: 0.0329

134304/135085 [============================>.] - ETA: 0s - loss: 13.3189 - acc: 0.0328

135085/135085 [==============================] - 6s 43us/step - loss: 13.3194 - acc: 0.0328 - val_loss: 13.6716 - val_acc: 1.3324e-04
